In [1]:
from IPython.display import display
from IPython.display import HTML
import IPython.core.display as di # Example: di.display_html('<h3>%s:</h3>' % str, raw=True)

# This line will hide code by default when the notebook is exported as HTML
# di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

display(HTML("<style>.container { width:100% !important; }</style>"))


import hail as hl
hl.init(tmp_dir='/net/scratch/people/plggosborcz')

Running on Apache Spark version 2.4.3
SparkUI available at http://p0586.prometheus:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.30-2ae07d872f43
LOGGING: writing to /net/archive/groups/plggneuromol/GTS-analysis/analysis/hail-20200930-1929-0.2.30-2ae07d872f43.log


In [2]:
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
from hail.plot import show
from pprint import pprint
from bokeh.layouts import gridplot
hl.plot.output_notebook()

import numpy as np
import pandas as pd
from functools import reduce
from itertools import chain
import statistics as stat

Loading BokehJS ...

In [4]:
from bokeh.plotting import figure, show, output_notebook
output_notebook()

Loading BokehJS ...

In [5]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/GTS-analysis/data/GTS-with-pheno.mt')

In [5]:
mt.count()

(16233065, 185)

In [17]:
cd = hl.import_table('/net/archive/groups/plggneuromol/GTS-analysis/cadd/whole_genome_SNVs.tsv.gz', force_bgz = True, delimiter='\t', comment='#', 
                     types = {'f0':'tstr', 'f1':'tint', 'f2':'tstr', 'f3':'tstr', 'f4':'tfloat', 'f5':'tfloat'},
                     no_header = True)

2020-09-08 10:30:03 Hail: INFO: Reading table with no type imputation
  Loading column 'f0' as type 'str' (user-specified)
  Loading column 'f1' as type 'int32' (user-specified)
  Loading column 'f2' as type 'str' (user-specified)
  Loading column 'f3' as type 'str' (user-specified)
  Loading column 'f4' as type 'float64' (user-specified)
  Loading column 'f5' as type 'float64' (user-specified)



In [18]:
cd = cd.select(variants = hl.parse_variant('chr'+hl.str(cd.f0)+':'+hl.str(cd.f1)+':'+hl.str(cd.f2)+':'+hl.str(cd.f3), reference_genome='GRCh38'),
               cadd_score = cd.f5)

In [24]:
cd = hl.read_table('/net/archive/groups/plggneuromol/GTS-analysis/data/cadd-all.mt')

In [6]:
contigs = ['chr1',
 'chr2',
 'chr3',
 'chr4',
 'chr5',
 'chr6',
 'chr7',
 'chr8',
 'chr9',
 'chr10',
 'chr11',
 'chr12',
 'chr13',
 'chr14',
 'chr15',
 'chr16',
 'chr17',
 'chr18',
 'chr19',
 'chr20',
 'chr21',
 'chr22']

In [7]:
contigs

['chr1',
 'chr2',
 'chr3',
 'chr4',
 'chr5',
 'chr6',
 'chr7',
 'chr8',
 'chr9',
 'chr10',
 'chr11',
 'chr12',
 'chr13',
 'chr14',
 'chr15',
 'chr16',
 'chr17',
 'chr18',
 'chr19',
 'chr20',
 'chr21',
 'chr22']

In [46]:
for c in contigs:
    cd_chr = cd.filter(cd.variants.locus.contig == c)
    cd_chr = cd_chr.key_by(cd_chr.variants.locus,cd_chr.variants.alleles)
    cd_chr.write('/net/archive/groups/plggneuromol/GTS-analysis/data/cadd-per-chr/'+c+'_cadd.mt')
    
cd_chr = cd.filter(hl.array(contigs).contains(cd.variants.locus.contig), keep = False)
cd_chr = cd_chr.key_by(cd_chr.variants.locus,cd_chr.variants.alleles)
cd_chr.write('/net/archive/groups/plggneuromol/GTS-analysis/data/cadd-per-chr/other_chroms_cadd.mt')

2020-09-08 16:00:18 Hail: INFO: Coerced sorted dataset
2020-09-08 16:01:04 Hail: INFO: wrote table with 691443036 rows in 204 partitions to /net/archive/groups/plggneuromol/GTS-analysis/data/cadd-per-chr/chr1_cadd.mt
2020-09-08 16:01:39 Hail: INFO: Coerced sorted dataset
2020-09-08 16:02:24 Hail: INFO: wrote table with 721644684 rows in 214 partitions to /net/archive/groups/plggneuromol/GTS-analysis/data/cadd-per-chr/chr2_cadd.mt
2020-09-08 16:02:58 Hail: INFO: Coerced sorted dataset
2020-09-08 16:03:41 Hail: INFO: wrote table with 594300405 rows in 176 partitions to /net/archive/groups/plggneuromol/GTS-analysis/data/cadd-per-chr/chr3_cadd.mt
2020-09-08 16:04:14 Hail: INFO: Coerced sorted dataset
2020-09-08 16:04:59 Hail: INFO: wrote table with 569258001 rows in 169 partitions to /net/archive/groups/plggneuromol/GTS-analysis/data/cadd-per-chr/chr4_cadd.mt
2020-09-08 16:05:34 Hail: INFO: Coerced sorted dataset
2020-09-08 16:06:16 Hail: INFO: wrote table with 543796134 rows in 160 partit

In [8]:
#do the same for mts
for c in contigs:
    mt_chr = mt.filter_rows(mt.locus.contig == c)
    cd = hl.read_table('/net/archive/groups/plggneuromol/GTS-analysis/data/cadd-per-chr/'+c+'_cadd.mt')
    
    mt_chr = mt_chr.annotate_rows(cadd = cd[mt_chr.row_key])
    mt_chr.write('/net/archive/groups/plggneuromol/GTS-analysis/data/mt-annotated-per-chr/'+c+'-anno.mt')

2020-09-09 20:38:50 Hail: INFO: wrote matrix table with 1343586 rows and 185 columns in 127 partitions to /net/archive/groups/plggneuromol/GTS-analysis/data/mt-annotated-per-chr/chr1-anno.mt
2020-09-09 20:39:16 Hail: INFO: wrote matrix table with 1485928 rows and 185 columns in 124 partitions to /net/archive/groups/plggneuromol/GTS-analysis/data/mt-annotated-per-chr/chr2-anno.mt
2020-09-09 20:39:40 Hail: INFO: wrote matrix table with 1248675 rows and 185 columns in 106 partitions to /net/archive/groups/plggneuromol/GTS-analysis/data/mt-annotated-per-chr/chr3-anno.mt
2020-09-09 20:40:03 Hail: INFO: wrote matrix table with 1020630 rows and 185 columns in 95 partitions to /net/archive/groups/plggneuromol/GTS-analysis/data/mt-annotated-per-chr/chr4-anno.mt
2020-09-09 20:40:27 Hail: INFO: wrote matrix table with 1046234 rows and 185 columns in 90 partitions to /net/archive/groups/plggneuromol/GTS-analysis/data/mt-annotated-per-chr/chr5-anno.mt
2020-09-09 20:40:48 Hail: INFO: wrote matrix ta

In [9]:
cd = hl.read_table('/net/archive/groups/plggneuromol/GTS-analysis/data/cadd-per-chr/other_chroms_cadd.mt')

mt_chr = mt.filter_rows(hl.array(contigs).contains(mt.locus.contig), keep = False)

mt_chr = mt_chr.annotate_rows(cadd =cd[mt_chr.row_key])

mt_chr.write('/net/archive/groups/plggneuromol/GTS-analysis/data/mt-annotated-per-chr/other_chroms-anno.mt')

2020-09-09 20:45:55 Hail: INFO: wrote matrix table with 769742 rows and 185 columns in 1604 partitions to /net/archive/groups/plggneuromol/GTS-analysis/data/mt-annotated-per-chr/other_chroms-anno.mt


In [10]:
#now read all the matrices and join
mts = !ls /net/archive/groups/plggneuromol/GTS-analysis/data/mt-annotated-per-chr

In [12]:
mts

['chr10-anno.mt',
 'chr11-anno.mt',
 'chr12-anno.mt',
 'chr13-anno.mt',
 'chr14-anno.mt',
 'chr15-anno.mt',
 'chr16-anno.mt',
 'chr17-anno.mt',
 'chr18-anno.mt',
 'chr19-anno.mt',
 'chr1-anno.mt',
 'chr20-anno.mt',
 'chr21-anno.mt',
 'chr22-anno.mt',
 'chr2-anno.mt',
 'chr3-anno.mt',
 'chr4-anno.mt',
 'chr5-anno.mt',
 'chr6-anno.mt',
 'chr7-anno.mt',
 'chr8-anno.mt',
 'chr9-anno.mt',
 'other_chroms-anno.mt']

In [13]:
mt_chr = []

for m in mts:
    mt_chr.append(hl.read_matrix_table('/net/archive/groups/plggneuromol/GTS-analysis/data/mt-annotated-per-chr/'+m)) 

In [15]:
mt = hl.MatrixTable.union_rows(*mt_chr)

In [16]:
mt = mt.transmute_rows(cadd = mt.cadd.cadd_score)

In [17]:
mt.count()

(16233065, 185)

In [18]:
mt.write('/net/archive/groups/plggneuromol/GTS-analysis/data/GTS-pheno-cadd.mt')

2020-09-09 20:48:25 Hail: INFO: wrote matrix table with 16233065 rows and 185 columns in 1622 partitions to /net/archive/groups/plggneuromol/GTS-analysis/data/GTS-pheno-cadd.mt


In [62]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/GTS-analysis/data/GTS-pheno-cadd.mt')

In [63]:
mt.count()

(16233065, 185)

## join with gnomad

In [64]:
gd = hl.read_matrix_table('/net/archive/groups/plggneuromol/GTS-analysis/gnomad/gnomad-ready-to-join.ht')

In [65]:
#adjust column schema:

gd = gd.annotate_cols(s = hl.str(hl.str(gd.col_idx)+'gnomad'),
                      phenotypes = hl.struct(family = hl.str('gnomad'),
                                             sex = hl.str('gnomad'),
                                             kinship = hl.str('gnomad'),
                                             disease = hl.str('gnomad'),
                                             phenotype = hl.str('gnomad'),
                                             add_pheno = hl.str('gnomad'),
                                             heavy_tics = hl.str('gnomad')))
                      

In [66]:
gd = gd.key_cols_by(gd.s)

In [67]:
gd = gd.drop(gd.col_idx)

In [68]:
mt = mt.union_cols(gd, row_join_type='outer')

In [69]:
#filter out any rows that contain only refs:

mt = mt.filter_rows(hl.agg.any(mt.GT.is_non_ref()))

In [70]:
mt.write('/net/archive/groups/plggneuromol/GTS-analysis/data/GTS-gnomad.mt')

2020-09-11 18:50:48 Hail: INFO: wrote matrix table with 11825581 rows and 370 columns in 6622 partitions to /net/archive/groups/plggneuromol/GTS-analysis/data/GTS-gnomad.mt


### Perform PCA on subseted genotypes:

In [44]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/GTS-analysis/data/GTS-gnomad.mt')

In [7]:
subset = mt.sample_rows(0.001)

In [8]:
subset.count()

(11940, 370)

In [9]:
eigenvalues, pcs, _ = hl.hwe_normalized_pca(subset.GT)

2020-09-15 09:55:56 Hail: INFO: hwe_normalized_pca: running PCA using 11847 variants.
2020-09-15 09:56:08 Hail: INFO: pca: running PCA with 10 components...


In [10]:
mt = mt.annotate_cols(scores = pcs[mt.s].scores)

In [26]:
p = hl.plot.scatter(mt.scores[0],
                    mt.scores[1],
                    label=mt.phenotypes.family,
                    title='PCA', xlabel='PC1', ylabel='PC2')

In [27]:
show(p)

## annotate mt with nearest_genes_20kb



In [31]:
genes = hl.read_table('/net/archive/groups/plggneuromol/GTS-analysis/vcf_preprocessing/genecode_v32.ht')
genes = genes.filter(hl.is_valid_contig(genes['hg38.knownGene.chrom'], reference_genome='GRCh38'))


start = hl.cond(genes['hg38.knownGene.txStart'] < 20000, 1, genes['hg38.knownGene.txStart'] - 20000)
stop =  hl.cond(hl.contig_length(genes['hg38.knownGene.chrom'], reference_genome='GRCh38') - genes['hg38.knownGene.txEnd'] < 20000, 
                hl.contig_length(genes['hg38.knownGene.chrom'], reference_genome='GRCh38'),
                genes['hg38.knownGene.txEnd'] + 20000)

genes = genes.transmute(interval = 
                        hl.locus_interval(genes['hg38.knownGene.chrom'], 
                                          start,
                                          stop,
                                          reference_genome='GRCh38'))

genes = genes.key_by(genes.interval)

mt = mt.annotate_rows(nearest_genes_20kb = hl.array(hl.set(genes.index(mt.locus, all_matches=True)['hg38.kgXref.geneSymbol'])))

NameError: name 'mt' is not defined

In [47]:
mt.write('/net/archive/groups/plggneuromol/GTS-analysis/data/GTS-for-skat.mt')

2020-09-15 14:37:24 Hail: INFO: Ordering unsorted dataset with network shuffle
2020-09-15 14:38:24 Hail: INFO: wrote matrix table with 11825581 rows and 370 columns in 6622 partitions to /net/archive/groups/plggneuromol/GTS-analysis/data/GTS-for-skat.mt


## Make the simulated controls gender equal

In [71]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/GTS-analysis/data/GTS-for-skat.mt')

PAR coordinates in GrCh38: http://m.ensembl.org/info/genome/genebuild/human_PARS.html

In [72]:
par1 = [10001,2781479]
par2 = [155701383,156030895]

In [73]:
rg = hl.get_reference('GRCh38')

In [74]:
rg.lengths['chrX']

156040895

In [75]:
nonpars = [[1,10000],[2781480,155701382]]

### first see sex distribution in our samples

In [76]:
samples = mt.s.collect()

In [77]:
gnomad_females = samples[185:(185+70)] 

In [78]:
gnomads_males = samples[(185+70):]

In [80]:
mt = mt.transmute_cols(phenotypes = mt.phenotypes.annotate(sex = hl.cond(mt.s.contains('gnomad'),
                                                      hl.cond(hl.array(gnomad_females).contains(mt.s),
                                                            'F', 'M'),
                                                      mt.phenotypes['sex'])))

In [81]:
mt.aggregate_cols(hl.agg.counter(mt.phenotypes['sex']))

{'F': 140, 'M': 230}

In [82]:
mt = mt.annotate_entries(GT = hl.cond(
                         mt.locus.contig == 'chrX',
                         hl.cond(
                            mt.phenotypes['sex'] == 'F',
                            mt.GT,
                            hl.cond(
                                mt.locus.position > nonpars[1][1], 
                                mt.GT,
                                hl.cond(
                                    mt.locus.position < nonpars[0][1],
                                    hl.cond(
                                        mt.locus.position > nonpars[0][0],
                                        mt.GT,
                                        hl.call(mt.GT[0], mt.GT[0])),
                                    hl.call(mt.GT[0], mt.GT[0])))),
                         mt.GT))     

In [83]:
mt.write('/net/archive/groups/plggneuromol/GTS-analysis/data/GTS-gnomad-sex.mt')

2020-09-17 13:31:25 Hail: INFO: wrote matrix table with 11825581 rows and 370 columns in 6622 partitions to /net/archive/groups/plggneuromol/GTS-analysis/data/GTS-gnomad-sex.mt


### SKAT overrepresentation analysis:

In [5]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/GTS-analysis/data/GTS-gnomad-sex.mt')

In [6]:
## kinship cut-off 0.125 - fist compare gnomads with heavy tics, then test it on families.

def remove_sex_chrom(mtx):
    mtx = mtx.filter_rows(mtx.locus.contig != "chrY")
    return(mtx)


def remove_related(mtx, mtx_subset):
    pc_rel = hl.pc_relate(mtx_subset.GT, 0.001, k=2, statistics='kin')
    pairs = pc_rel.filter(pc_rel['kin'] > 0.125)
    related_samples_to_remove = hl.maximal_independent_set(pairs.i, pairs.j, keep=False)
    
    related_samples_to_remove = related_samples_to_remove.annotate(s = related_samples_to_remove.node.s)
    related_samples_to_remove = related_samples_to_remove.key_by('s')
    
    mtx = mtx.key_cols_by()
    mtx = mtx.filter_cols(hl.is_defined(related_samples_to_remove[mtx.s]), keep=False)
    mtx = mtx.key_cols_by(mtx.s)
    
    mtx_subset = mtx_subset.filter_cols(hl.is_defined(related_samples_to_remove[mtx_subset.s]), keep=False)
    
    return(mtx, mtx_subset)


def run_pca(mtx, mtx_subset):
    eigenvalues, pcs, _ = hl.hwe_normalized_pca(mtx_subset.GT)
    mtx = mtx.annotate_cols(scores = pcs[mtx.s].scores)
    p = hl.plot.scatter(mtx.scores[0],
                    mtx.scores[1],
                    label=mtx.phenotypes.family,
                    title='PCA', xlabel='PC1', ylabel='PC2')

    p2 = hl.plot.scatter(mtx.scores[2],
                        mtx.scores[3],
                        label=mtx.phenotypes.family,
                        title='PCA', xlabel='PC3', ylabel='PC4')

    p3 = hl.plot.scatter(mtx.scores[4],
                        mtx.scores[5],
                        label=mtx.phenotypes.family,
                        title='PCA', xlabel='PC5', ylabel='PC6')
    show(p)
    show(p2)
    show(p3)
    
    return(mtx)


def run_skat_log(mtx, gene_list, pcs): #does not have a family covariate
    
    mtx = mtx.filter_rows(hl.any(lambda x: hl.literal(gene_list).contains(x), mtx.nearest_genes_20kb))
    mtx = mtx.filter_rows(hl.agg.any(mtx.GT.is_non_ref()))
    mtx = mtx.explode_rows(mtx.nearest_genes_20kb)
    mtx = mtx.filter_rows(hl.literal(gene_list).contains(mtx.nearest_genes_20kb))
    
    
    scores = [mtx.scores[x] for x in list(range(pcs))]
                          
    
    skat_table = hl.skat(
                         key_expr=mtx.nearest_genes_20kb,
                         weight_expr=mtx.cadd,
                         y=mtx.category,
                         x=mtx.GT.n_alt_alleles(),
                         covariates=[1] + scores,
                         max_size = 2500,
                         logistic = True)
    
    genes_result = skat_table.filter(skat_table.p_value < 0.05/len(gene_list)).id.collect() 

    skat_table.filter(skat_table.p_value < 0.002).show(20)

    skat_table = skat_table.annotate(label = hl.literal(genes).contains(skat_table.id))

    qq_plot = hl.plot.qq(skat_table.p_value,
                                         label = skat_table.label,
                                         n_divisions = len(gene_list))
    show(qq_plot)
    
    return(skat_table, genes_result, qq_plot)


def full_skat_log(mtx, mtx_subset, gene_list, pcs):
    
    mtx = remove_sex_chrom(mtx)
    mtx_subset = remove_sex_chrom(mtx_subset)
    
    mtx, mtx_subset = remove_related(mtx, mtx_subset)
    
    mtx = run_pca(mtx, mtx_subset) #this matrix will be returned, so I can do SKAT with other list and parameters
    
    skat_table, genes_result, qq_plot = run_skat_log(mtx, gene_list, pcs)
    
    return(mtx, skat_table, genes_result, qq_plot)

In [7]:
mt = mt.filter_rows(mt.cadd > 0) # this is so I don't do too many tests

In [8]:
mt_subset = mt.sample_rows(0.002)

In [91]:
# mt_subset.write('/net/archive/groups/plggneuromol/GTS-analysis/data/GTS-gnomad-test.mt')

2020-09-17 13:47:56 Hail: INFO: wrote matrix table with 12825 rows and 370 columns in 6622 partitions to /net/archive/groups/plggneuromol/GTS-analysis/data/GTS-gnomad-test.mt


In [9]:
mt_subset = hl.read_matrix_table('/net/archive/groups/plggneuromol/GTS-analysis/data/GTS-gnomad-test.mt')
mt_subset = mt_subset.filter_rows(mt_subset.cadd > 0) # this is so I don't do too many tests

In [9]:
genes = ['DCC', 'RBFOX', 'SLC30A9', 'DCAF4L1', 'SORCS3', 'KCNQ5', 'KCNQ-IT1', 'APOPT1', 'C14orf2', 'NAA11', 'NEGR1',
        'CHADL', 'SOX5'] # the other gene from 'L3MBTL2' was deleted


GTS_genes = ['PANK2', 'COL27A1', 'PDGFB', 'CELSR3', 'OPA1', 'FBN2', 'WWC1', 'NIPBL', 
             'FN1', 'FBN2', 'SLITRK1', 'SLITRK2', 'SLITRK3', 'SLITRK4', 'SLITRK5', 'SLITRK6', 
             'HDC', 'OPRK1', 'PCDH10', 'NTSR2', 'OPRK1', 'CHD8', 'SCUBE1', 'PNKD', 'CNTNAP2', 'MOG', 
             'DRD2', 'DRD3', 'DRD4', 'DRD5', 'DAT1', 'DBH', 'HTR2A', 'TPH2', 'EAAT1', 'SAPAP3',
            'CTNNA3', 'NLGN4', 'FSCB', 'IMMP2L', 'NRXN1', 'AADAC', 'DBH', 'MAOA', 'HTR1A', 'HTR2C', 'SLC6A4',
             'TPH2', 'COL27A1', '5-HTTLPR', 'EAAT1', 'COL8A1', 'KCNE1', 'KCNE2'] # a list manually curated from literature


allgenes = hl.import_table('/net/archive/groups/plggneuromol/GTS-analysis/gts_gene_lists/human-genes-with-GO-and-symbols') 
allgenes = allgenes.select('UniProtKB Gene Name symbol')

allgenes = allgenes.filter(allgenes['UniProtKB Gene Name symbol'] != "")


allgenes = allgenes['UniProtKB Gene Name symbol'].collect()


genes_scores = genes + GTS_genes

2020-09-30 19:31:01 Hail: INFO: Reading table with no type imputation
  Loading column 'Gene stable ID' as type 'str' (type not specified)
  Loading column 'UniProtKB Gene Name symbol' as type 'str' (type not specified)



run logistic skat on individuals outside of families with heavy tics

In [11]:
mt = mt.annotate_cols(category = hl.cond(mt.s.contains('gnomad'), False, (mt.phenotypes.disease == 'YES')))

In [12]:
mt_subset = mt_subset.annotate_cols(category = hl.cond(mt_subset.s.contains('gnomad'), False, (mt_subset.phenotypes.disease == 'YES')))

In [13]:
mt = mt.filter_cols((mt.s.contains('gnomad')) | ((mt.phenotypes.family =='.') & (mt.phenotypes.disease =='YES')))
mt_subset = mt_subset.filter_cols((mt_subset.s.contains('gnomad')) | ((mt_subset.phenotypes.family =='.') & (mt_subset.phenotypes.disease =='YES')))

In [14]:
samples = mt.s.collect()

In [15]:
mt.aggregate_cols(hl.agg.filter(hl.array(samples[0:39]).contains(mt.s),hl.agg.counter(mt.phenotypes.sex)))

#first 70 gnomad samples are females

{'F': 5, 'M': 34}

In [16]:
to_keep = samples[0:44] + samples[110:144]

In [17]:
mt = mt.filter_cols(hl.array(to_keep).contains(mt.s)) # filter out excessive gnomads
mt_subset = mt_subset.filter_cols(hl.array(to_keep).contains(mt_subset.s))

In [18]:
#after PCA - the samples : WGS_113 and WGS_108 are removed, we also remove two gnomads for balance:

mt = mt.filter_cols((mt.s != 'WGS_113') & (mt.s != 'WGS_108') & (mt.s != '4gnomad') & (mt.s != '104gnomad'))

In [19]:
mt_subset = mt_subset.filter_cols((mt_subset.s != 'WGS_113') & (mt_subset.s != 'WGS_108') & (mt_subset.s != '4gnomad') & (mt_subset.s != '104gnomad'))

In [20]:
groups = mt.aggregate_cols(hl.agg.counter(mt.phenotypes.sex))

In [21]:
groups

{'F': 8, 'M': 66}

In [22]:
mt, skat_table_log, genes_result_log, qq_plot_log = full_skat_log(mt, mt_subset, genes_scores, 3)

2020-09-21 12:21:34 Hail: INFO: hwe_normalized_pca: running PCA using 9907 variants.
2020-09-21 12:23:02 Hail: INFO: pca: running PCA with 2 components...
2020-09-21 12:46:46 Hail: INFO: Wrote all 4 blocks of 12812 x 74 matrix with block size 4096.
2020-09-21 12:46:48 Hail: INFO: wrote matrix with 74 rows and 74 columns as 1 block of size 4096 to file:/net/scratch/people/plggosborcz/hail.HZdaT0l7EvQI/eKxIpfEhbv.bm
2020-09-21 12:46:48 Hail: INFO: Ordering unsorted dataset with network shuffle
2020-09-21 12:46:51 Hail: INFO: wrote table with 0 rows in 1 partition to file:/net/scratch/people/plggosborcz/hail.HZdaT0l7EvQI/U6ogqvUTiQ
2020-09-21 12:46:51 Hail: WARN: cols(): Resulting column table is sorted by 'col_key'.
    To preserve matrix table column order, first unkey columns with 'key_cols_by()'
2020-09-21 12:47:11 Hail: INFO: hwe_normalized_pca: running PCA using 9907 variants.
2020-09-21 12:47:32 Hail: INFO: pca: running PCA with 10 components...
2020-09-21 12:48:13 Hail: INFO: Coer

2020-09-21 12:48:16 Hail: INFO: Coerced sorted dataset
2020-09-21 12:52:25 Hail: INFO: Coerced sorted dataset
2020-09-21 12:52:25 Hail: INFO: Coerced dataset with out-of-order partitions.
2020-09-21 12:52:25 Hail: INFO: Coerced sorted dataset
2020-09-21 12:55:51 Hail: INFO: Coerced sorted dataset
2020-09-21 12:55:51 Hail: INFO: Coerced dataset with out-of-order partitions.


id,size,q_stat,p_value,fault
str,int32,float64,float64,int32
"""CHADL""",98,1.43e+03,1.69e-03,0


2020-09-21 12:55:54 Hail: INFO: Coerced sorted dataset
2020-09-21 12:59:12 Hail: INFO: Coerced sorted dataset
2020-09-21 12:59:12 Hail: INFO: Coerced dataset with out-of-order partitions.
2020-09-21 12:59:12 Hail: INFO: Coerced sorted dataset
2020-09-21 12:59:12 Hail: INFO: Coerced dataset with out-of-order partitions.


In [42]:
show(qq_plot_log)

In [ ]:
#skat_table_log.write('/net/archive/groups/plggneuromol/GTS-analysis/data/skat.ht') # 2 PCs two samples deleted from the analysis

In [7]:
skat_table_log.write('/net/archive/groups/plggneuromol/GTS-analysis/data/skat-3pcs.ht') # 2 PCs two samples deleted from the analysis

NameError: name 'skat_table_log' is not defined

### Collect the number of variants in control individuals for the classifier

A few iterations over various numbers of genes and CADD score cutt-offs were run

In [5]:
#skat = hl.read_table('/net/archive/groups/plggneuromol/GTS-analysis/data/skat.ht')
skat = hl.read_table('/net/archive/groups/plggneuromol/GTS-analysis/data/skat-3pcs.ht')

skat.order_by('p_value').show(20)

id,size,q_stat,p_value,fault,label
str,int32,float64,float64,int32,bool
"""CHADL""",98,1.43e+03,1.69e-03,0,true
"""PCDH10""",240,1.50e+03,7.37e-03,0,false
"""MAOA""",92,1.10e+03,9.09e-03,0,false
"""KCNQ5""",1147,5.28e+03,1.04e-02,0,true
"""DRD2""",223,1.07e+03,1.21e-02,0,false
"""PDGFB""",173,7.98e+02,2.32e-02,0,false
"""SLITRK2""",78,5.16e+02,2.67e-02,0,false
"""HTR2C""",306,2.34e+03,2.82e-02,0,false
"""SLITRK4""",82,1.20e+03,3.87e-02,0,false


In [6]:
#reimport the matrixtable again

mt_test = hl.read_matrix_table('/net/archive/groups/plggneuromol/GTS-analysis/data/GTS-gnomad-sex.mt')
mt_test = mt_test.filter_rows(mt_test.cadd > 0) # this is so I don't do too many tests

mt_test = mt_test.annotate_cols(category = hl.cond(mt_test.s.contains('gnomad'), False, (mt_test.phenotypes.disease == 'YES'))) #logistic
mt_test = mt_test.filter_cols(mt_test.phenotypes.family == '.', keep = False)

mt_test = mt_test.filter_cols((mt_test.phenotypes.disease == 'YES') | (mt_test.phenotypes.disease == 'NO'))
mt_test.count()

(6437977, 144)

In [ ]:
controls = np.zeros((len([2,3,5,10,15]), len([10,11,15,18])))
gts = np.zeros((len([2,3,5,10,15]), len([10,11,15,18])))

variants_controls = np.zeros((len([2,3,5,10,15]), len([10,11,15,18])))
variants_gts = np.zeros((len([2,3,5,10,15]), len([10,11,15,18])))

controls_test = np.zeros((len([2,3,5,10,15]), len([10,11,15,18])))
gts_test = np.zeros((len([2,3,5,10,15]), len([10,11,15,18])))


for rows, n in enumerate([2,3,5,10,15]):
    top_log = hl.array(skat.order_by('p_value').id.take(n))
    mt_skat_log_40 = mt.filter_rows(hl.any(lambda x: hl.literal(top_log).contains(x), mt.nearest_genes_20kb)) # to ma gnomadów i heavy tics
    mt_test_skat_log_40 = mt_test.filter_rows(hl.any(lambda x: hl.literal(top_log).contains(x), mt_test.nearest_genes_20kb)) # to ma rodziny
    
    for cols, c in enumerate([10,11,15,18]):

        mt_skat_log_40 = mt_skat_log_40.filter_rows(mt_skat_log_40.cadd > c)
        skat_log_40_res = mt_skat_log_40.annotate_cols(non_refs = hl.agg.count_where(mt_skat_log_40.GT.is_non_ref())) #count variants per sample of gnomads and heavy tics
        skat_log_40_sum = skat_log_40_res.cols()

        skat_log_40_sum = (skat_log_40_sum.group_by(skat_log_40_sum.category).aggregate(mean_non_refs = hl.agg.mean(skat_log_40_sum.non_refs)))

        results = skat_log_40_sum.mean_non_refs.collect()
        ctrl = results[0]

        variants_controls[rows, cols] = results[0]
        variants_gts[rows, cols] = results[1]


        test = skat_log_40_res.cols()
        test = test.annotate(test_result = (((hl.float(ctrl) - test.non_refs) / hl.float(ctrl)) < -0.10))
        test = test.annotate(success = (test.test_result == test.category))
        test2 = test.filter(test.s.contains('gnomad'))
        test1 = test.filter(test.phenotypes.disease == 'YES')

        try:
            controls[rows, cols] = test2.aggregate(hl.agg.counter(test2.success))[1]
        except KeyError:
            controls[rows, cols] = 0

        try:
            gts[rows, cols] = test1.aggregate(hl.agg.counter(test1.success))[1]
        except KeyError:
            gts[rows, cols] = 0
                
        mt_test_skat_log_40 = mt_test_skat_log_40.filter_rows(mt_test_skat_log_40.cadd > c)
        skat_log_40_res = mt_test_skat_log_40.annotate_cols(non_refs = hl.agg.count_where(mt_test_skat_log_40.GT.is_non_ref())) #count variants per sample
        
        test3 = skat_log_40_res.cols()
        test3 = test3.annotate(test_result = (((hl.float(ctrl) - test3.non_refs) / hl.float(ctrl)) < -0.10))
        test3 = test3.annotate(success = (test3.test_result == test3.category))
        test4 = test3.filter(test3.phenotypes.disease == 'NO')
        test5 = test3.filter(test3.phenotypes.disease == 'YES')
            
        try:
            controls_test[rows, cols] = test4.aggregate(hl.agg.counter(test4.success))[1]
        except KeyError:
            controls_test[rows, cols] = 0
            
        try:
            gts_test[rows, cols] = test5.aggregate(hl.agg.counter(test5.success))[1]
        except KeyError:
            gts_test[rows, cols] = 0

In [70]:
#np.save('gts_new', gts)
#np.save('controls_new', controls)


#np.save('variants_gts_new', variants_gts)
#np.save('variants_controls_new', variants_controls)


#np.save('gts_test_new', gts_test)
#np.save('controls_test_new', controls_test)

In [ ]:
mt_test.filter_cols(mt_test.phenotypes.disease == 'YES').count()

In [ ]:
mt_test.filter_cols(mt_test.phenotypes.disease == 'NO').count()

In [11]:
a = list((controls_test*100/53).flatten())
b = list((gts_test*100/91).flatten())

np.where((controls_test*100/53) + (gts_test*100/91) > 112)

(array([0, 1, 2, 2]), array([0, 0, 1, 3]))

from the new arrays I will use the following variants to generate te AUC:

2,1

In [12]:
from bokeh.plotting import figure, output_notebook, show

output_notebook

p2 = figure(plot_width=600, plot_height=600, x_range=(0, 100), y_range=(0, 100))

# add a circle renderer with a size, color, and alpha
p2.circle(a, b, size=10, alpha=0.5)
p2.circle((controls_test*100/53)[1,1], (gts_test*100/91)[1,1], size=15, alpha=1, color="orange")
p2.circle((controls_test*100/53)[1,0], (gts_test*100/91)[1,0], size=15, alpha=1, color="orange")
p2.circle((controls_test*100/53)[2,1], (gts_test*100/91)[2,1], size=15, alpha=1, color="orange")

p2.line([100, 0], [0, 100], line_width=0.5, color="lightblue")



p2.xaxis.axis_label = '% of controls assigned corectly'
p2.yaxis.axis_label = '% of gts assigned corectly'

p2.yaxis.axis_label_text_font_size = "25px"
p2.xaxis.axis_label_text_font_size = "25px"

# show the results
show(p2)

#this graph shows % of well asigned controls (x asis) vs well asigned GTS (y axis)



### ROC

In [13]:
gts = np.load('gts_new.npy')
controls = np.load('controls_new.npy')


variants_gts = np.load('variants_gts_new.npy')
variants_controls = np.load('variants_controls_new.npy')


gts_test = np.load('gts_test_new.npy')
controls_test = np.load('controls_test_new.npy')

In [8]:
categories = mt_test.category.collect()

In [234]:
top_log = hl.array(skat.order_by('p_value').id.take(5))

In [276]:
#2,1

false_pos = []
true_pos = []

top_log = hl.array(skat.order_by('p_value').id.take(5))
mt_test_skat_log = mt_test.filter_rows(hl.any(lambda x: hl.literal(top_log).contains(x), mt_test.nearest_genes_20kb))
mt_test_skat_log = mt_test_skat_log.filter_rows(mt_test_skat_log.cadd > 11)
mt_test_skat_log = mt_test_skat_log.annotate_cols(non_refs = hl.agg.count_where(mt_test_skat_log.GT.is_non_ref())) #count variants per sample

non_refs = mt_test_skat_log.non_refs.collect()

for x in np.linspace(-0.4,1,100):
    test_results = (variants_controls[2,1] - non_refs) / variants_controls[2,1] 
    
    asignment = (test_results < x)
    
    try:
        false_pos.append(sum(asignment[np.invert(categories)])/53)

    except(KeyError, TypeError):

        false_pos.append(0)
            
    try:
        true_pos.append(sum(asignment[categories])/91)

    except(KeyError, TypeError):
        true_pos.append(0)
        



In [295]:
mt_test_skat_log.count()

(180, 144)

In [278]:
hl.eval(top_log)

['CHADL', 'PCDH10', 'MAOA', 'KCNQ5', 'DRD2']

In [284]:
np.save('false_pos', false_pos)
np.save('true_pos', true_pos)

In [14]:
false_pos = np.load('false_pos.npy')
true_pos = np.load('true_pos.npy')

In [15]:
y = np.linspace(0,1,10)
x = np.linspace(0,1,10)

In [16]:
from bokeh.plotting import figure, output_notebook, show

output_notebook

p5 = figure(plot_width=800, plot_height=800)


p5.line(x, y, line_width=4, line_color='lightgrey')
p5.line(false_pos, true_pos, line_width=4, alpha=0.5)

p5.xaxis.axis_label = 'false positives'
p5.yaxis.axis_label = 'true positives'

p5.yaxis.axis_label_text_font_size = "25px"
p5.xaxis.axis_label_text_font_size = "25px"



# show the results
show(p5)

In [17]:
auc = np.trapz(true_pos, false_pos)

auc

0.6210864607091022

Get the false positive rate of models:

In [18]:
mt_test = mt_test.filter_rows(mt_test.cadd > 11)

In [19]:
genes_background = mt_test.nearest_genes_20kb.collect()

In [20]:
genes_background_1 =  set([val for sublist in genes_background for val in sublist])

In [21]:
len(genes_background_1)

54392

In [22]:
len(allgenes)

22845

In [23]:
genes_background = [x for x in genes_background_1 if x in allgenes]

len(genes_background)

18036

In [20]:
mt_test.write('/net/archive/groups/plggneuromol/GTS-analysis/data/mt_test.mt')

2020-09-23 18:41:04 Hail: INFO: wrote matrix table with 491634 rows and 144 columns in 6622 partitions to /net/archive/groups/plggneuromol/GTS-analysis/data/mt_test.mt


In [24]:
mt_test = hl.read_matrix_table('/net/archive/groups/plggneuromol/GTS-analysis/data/mt_test.mt')

In [25]:
variants_controls

array([[ 13.59459459,   7.81081081,   1.75675676,   0.75675676],
       [ 13.75675676,   7.97297297,   1.91891892,   0.91891892],
       [ 45.48648649,  35.2972973 ,  11.54054054,   6.08108108],
       [ 58.        ,  46.45945946,  14.51351351,   7.10810811],
       [103.89189189,  85.7027027 ,  32.27027027,  13.56756757]])

In [26]:
false_pos_test = np.zeros((len(range(0,1000)), len(np.linspace(-1.5,1.5,50))))
true_pos_test = np.zeros((len(range(0,1000)), len(np.linspace(-1.5,1.5,50))))

In [27]:
categories = mt_test.category.collect()

In [28]:
for gene in range(0,1000):

    randoms = np.random.randint(len(genes_background), size=5)
    top_log = [j for i, j in enumerate(genes_background) if i in randoms]

    skat_log_res = mt_test.filter_rows(hl.any(lambda x: hl.literal(top_log).contains(x), mt_test.nearest_genes_20kb))

    skat_log_res = skat_log_res.annotate_cols(non_refs = hl.agg.count_where(skat_log_res.GT.is_non_ref()))

    non_refs = skat_log_res.non_refs.collect()

    for step, z in enumerate(np.linspace(-1.5,1.5,50)):

        test_results = (variants_controls[2,1] - non_refs) / variants_controls[2,1] 
        asignment = (test_results < z)
        
        try:
            false_pos_test[gene, step] = sum(asignment[np.invert(categories)])/53

        except(KeyError, TypeError):

            false_pos_test[gene, step] = 0
            
        try:
            true_pos_test[gene, step] = sum(asignment[categories])/91

        except(KeyError, TypeError):
            true_pos_test[gene, step] = 0
        
    
    print('I have completed iteration number: ' + str(gene))

I have completed iteration number: 0
I have completed iteration number: 1
I have completed iteration number: 2
I have completed iteration number: 3
I have completed iteration number: 4
I have completed iteration number: 5
I have completed iteration number: 6
I have completed iteration number: 7
I have completed iteration number: 8
I have completed iteration number: 9
I have completed iteration number: 10
I have completed iteration number: 11
I have completed iteration number: 12
I have completed iteration number: 13
I have completed iteration number: 14
I have completed iteration number: 15
I have completed iteration number: 16
I have completed iteration number: 17
I have completed iteration number: 18
I have completed iteration number: 19
I have completed iteration number: 20
I have completed iteration number: 21
I have completed iteration number: 22
I have completed iteration number: 23
I have completed iteration number: 24
I have completed iteration number: 25
I have completed itera

I have completed iteration number: 213
I have completed iteration number: 214
I have completed iteration number: 215
I have completed iteration number: 216
I have completed iteration number: 217
I have completed iteration number: 218
I have completed iteration number: 219
I have completed iteration number: 220
I have completed iteration number: 221
I have completed iteration number: 222
I have completed iteration number: 223
I have completed iteration number: 224
I have completed iteration number: 225
I have completed iteration number: 226
I have completed iteration number: 227
I have completed iteration number: 228
I have completed iteration number: 229
I have completed iteration number: 230
I have completed iteration number: 231
I have completed iteration number: 232
I have completed iteration number: 233
I have completed iteration number: 234
I have completed iteration number: 235
I have completed iteration number: 236
I have completed iteration number: 237
I have completed iteratio

I have completed iteration number: 424
I have completed iteration number: 425
I have completed iteration number: 426
I have completed iteration number: 427
I have completed iteration number: 428
I have completed iteration number: 429
I have completed iteration number: 430
I have completed iteration number: 431
I have completed iteration number: 432
I have completed iteration number: 433
I have completed iteration number: 434
I have completed iteration number: 435
I have completed iteration number: 436
I have completed iteration number: 437
I have completed iteration number: 438
I have completed iteration number: 439
I have completed iteration number: 440
I have completed iteration number: 441
I have completed iteration number: 442
I have completed iteration number: 443
I have completed iteration number: 444
I have completed iteration number: 445
I have completed iteration number: 446
I have completed iteration number: 447
I have completed iteration number: 448
I have completed iteratio

I have completed iteration number: 635
I have completed iteration number: 636
I have completed iteration number: 637
I have completed iteration number: 638
I have completed iteration number: 639
I have completed iteration number: 640
I have completed iteration number: 641
I have completed iteration number: 642
I have completed iteration number: 643
I have completed iteration number: 644
I have completed iteration number: 645
I have completed iteration number: 646
I have completed iteration number: 647
I have completed iteration number: 648
I have completed iteration number: 649
I have completed iteration number: 650
I have completed iteration number: 651
I have completed iteration number: 652
I have completed iteration number: 653
I have completed iteration number: 654
I have completed iteration number: 655
I have completed iteration number: 656
I have completed iteration number: 657
I have completed iteration number: 658
I have completed iteration number: 659
I have completed iteratio

I have completed iteration number: 846
I have completed iteration number: 847
I have completed iteration number: 848
I have completed iteration number: 849
I have completed iteration number: 850
I have completed iteration number: 851
I have completed iteration number: 852
I have completed iteration number: 853
I have completed iteration number: 854
I have completed iteration number: 855
I have completed iteration number: 856
I have completed iteration number: 857
I have completed iteration number: 858
I have completed iteration number: 859
I have completed iteration number: 860
I have completed iteration number: 861
I have completed iteration number: 862
I have completed iteration number: 863
I have completed iteration number: 864
I have completed iteration number: 865
I have completed iteration number: 866
I have completed iteration number: 867
I have completed iteration number: 868
I have completed iteration number: 869
I have completed iteration number: 870
I have completed iteratio

sum(asignment[np.invert(categories)]) #number of false positives
sum(np.invert(asignment)[categories]) # number of false negatives
        
sum(np.invert(asignment)[np.invert(categories)]) #number of true negatives
sum(asignment[categories]) #number of true positives 

In [29]:
np.save('false_pos_test', false_pos_test)
np.save('true_pos_test', true_pos_test)

In [30]:
false_pos_test = np.load('false_pos_test.npy')
true_pos_test = np.load('true_pos_test.npy')

## AUC

In [46]:
aucs = []
for i in range(0,1000):
    aucs.append(np.trapz(true_pos_test[i,], false_pos_test[i,]))

In [39]:
np.percentile(aucs, 99.9)

0.610516276176654

In [41]:
auc

0.6210864607091022

In [43]:
test1 = np.array(([0,1,2,4,5], [0,2,2,3,5]))
test2 = np.array(([1,1,1,1,1], [2,2,2,2,2]))

In [44]:
y = np.linspace(0,1,10)
x = np.linspace(0,1,10)

In [49]:
p4 = figure(plot_width=800, plot_height=800)
p4.line(x, y, line_width=4, line_color='lightgrey')


for i in range(0,1000):
    p4.line(false_pos_test[i,], true_pos_test[i,], line_width=0.5, alpha=0.25)
    
p4.line(false_pos, true_pos, line_width=4, line_color='orange')

p4.xaxis.axis_label = 'false positives'
p4.yaxis.axis_label = 'true positives'

p4.yaxis.axis_label_text_font_size = "25px"
p4.xaxis.axis_label_text_font_size = "25px"

# show the results
show(p4)